In [ ]:
import os
import glob
import mne
from mne import Report
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from utils import (get_parent_dir, extract_lab_sec, loc_of_sep_lines, plot_seizure_horizontal, plot_seizure_vertical,
                                        raw_to_df, extract_groups, extract_ordered_groups, save_plotly_to_html,
                                        shrink_df_to_tmax, create_results_folders, save_fig_to_disc, plot_interactive_subplot_with_table,
                                        extract_parameters_from_raw, plot_eventcounts, plot_interactive_tables)
import plotly as py
import ipywidgets as widgets
import plotly.io as pio
from plotly.subplots import make_subplots

py.offline.init_notebook_mode(connected=True)
graph_sep_line_width = 5
plot_tmin = 5
plot_tmax = 60

In [ ]:
def extract_parameters_from_raw(raw=None):
    highp = raw.info["highpass"]
    lowp = raw.info["lowpass"]
    sfreq = raw.info["sfreq"]
    aq = raw.info["meas_date"]
    channels = raw.info["ch_names"]
    nr_channels = raw.info["nchan"]
    return highp, lowp, sfreq, aq, channels, nr_channels

def raw_to_df(raw, edf=None):
    df = pd.DataFrame(raw.annotations)
    to_drop = ["duration"]
    df = df.drop(to_drop, axis=1)
    if "Beginn" in str(df["description"]):
        samp_beginn = df[df["description"].str.contains("Beginn")]["onset"]
        onset = samp_beginn
        if isinstance(samp_beginn, pd.core.series.Series):
            print(f"There are multiple markers for seizure onset in this file --> taking first one.")
            samp_beginn = samp_beginn.iloc[0]
            onset = samp_beginn
    else:
        print("Error: No marker containing \"Beginn\" found, cannot determine seizure onset for file: ", edf)
        print("Setting seizure onset to the beginning of the file")
        samp_beginn = 0
        onset = "No seizure onset was marked"
    df["time_from_onset"] = df["onset"] - float(samp_beginn)
    df = df.drop(["orig_time"], axis=1)
    return df, onset

In [ ]:
edfs = glob.glob("../data/*.edf")
e = edfs[0]
e

In [ ]:
raw = mne.io.read_raw(e)
df, onset = raw_to_df(raw, e)
e_events, s_events, t_events = extract_groups(df, e)
e_events

In [ ]:
def plot_eventcounts(df=None, eeg=None, semio=None, source=None):
    fig, ax = plt.subplots(figsize=(15,26))
    plt.suptitle(str(source) + " - Event counts")
    #EEG
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.set_title("EEG Events")
    if len(eeg['description'].value_counts()) > 0:
        sns.countplot(y="description", data=eeg, orient="h", ax=ax1, order = eeg['description'].value_counts().index)
    else:
        #plt.plot(x=1, y=1)
        ax1.set_title("No EEG events found in file...")
    #Semiology
    ax2 = fig.add_subplot(2, 1, 2)
    ax2.set_title("Semiology Events")
    if [semio['description'].value_counts()] != []:
        sns.countplot(y="description", data=semio, orient="h", ax=ax2, order = semio['description'].value_counts().index)
    else:
        plt.plot(x=1, y=1)
        ax2.set_title("No Semiology events found in file...")
    ax.set_yticks([])
    ax.set_axis_off()
    fig.subplots_adjust(hspace=0.2)
    return fig

fig = plot_eventcounts(df=df, eeg=e_events, semio=s_events, source=e)

In [ ]:
df.to_csv("df_template.csv")

----
# Start from template df
----

In [ ]:
df = pd.read_csv("df_template.csv")
e_events, s_events, t_events = extract_groups(df, "Test_Dataframe")
e = "test_folder"

In [ ]:
import plotly.express as px
def plot_interactive_eventcounts(eeg=None, semio=None, test=None, source=None):
    fig = make_subplots(rows=3, cols=1, 
                        specs=[[{"type": "bar"}],
                                [{"type": "bar"}],
                                [{"type": "bar"}]],
                        subplot_titles=("EEG events", "Semiology events", "Testing events"),
                        #row_width=[0.1, 0.1, 0.1],
                        vertical_spacing=0.2
                        )
    # EEG
    fig.add_trace(go.Histogram(y=e_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="EEG"),
                    row=1, col=1
                    )

    # Semio
    fig.add_trace(go.Histogram(y=s_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="Semiology"),
                    row=2, col=1
                    )    

    # Test
    fig.add_trace(go.Histogram(y=t_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="Testing"),
                    row=3, col=1
                    )


    fig.update_yaxes(categoryorder="total descending")
    fig.update_layout(width=1000, height=1200)
    return fig

plot_interactive_eventcounts(eeg=e_events, semio=s_events, test=t_events, source="Test")

In [ ]:
def plot_interactive_eeg_and_semio(eeg=None, semio=None, test=None, source=None):
    fig = make_subplots(rows=1, cols=2, start_cell="top-left",
                        subplot_titles=("EEG events", "Semiology events"),
                        #row_width=[0.1, 0.1, 0.1],
                        horizontal_spacing=0.2
                        )
    # EEG
    fig.add_trace(go.Histogram(y=e_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="EEG"),
                    row=1, col=1
                    )

    # Semio
    fig.add_trace(go.Histogram(y=s_events["description"], 
                        histfunc="count",
                        orientation="h",
                        name="Semiology"),
                    row=1, col=2
                    )    


    fig.update_yaxes(categoryorder="total descending")
    fig.update_layout(width=1100, height=800)
    return fig

plot_interactive_eventcounts(eeg=e_events, semio=s_events, test=t_events, source="Test")

In [ ]:
t_events

In [ ]:
def plot_interactive_eventcount(df=None, mode=None, source=None):
    fig = go.Figure(
        data=[go.Histogram(y=df["description"], 
                            histfunc="count",
                            orientation="h")]
                    )
    fig.update_yaxes(categoryorder="total descending")
    fig.update_layout(title=(source + " - " + mode + " - Eventcount"))
    return fig
plot_interactive_eventcount(t_events, mode="Semiology", source= "asd")

----
## Testing with failed/passed viz...
----

In [ ]:
def plot_interactive_testing_results(t_events=None, title="Testing results"):
    t_events_failed = t_events[t_events["description"].apply(lambda x: x.endswith("0"))]
    t_events_failed["description"] = t_events_failed.description.str.split("0").str[0]
    t_events_passed = t_events[t_events["description"].apply(lambda x: x.endswith("1"))]
    t_events_passed["description"] = t_events_passed.description.str.split("1").str[0]
    fig = go.Figure()

    # passed
    fig.add_trace(go.Scatter(x=t_events_passed["time_from_onset"], 
                        y=t_events_passed["description"],
                        name="passed",
                        mode="markers")
                    )

    # failed
    fig.add_trace(go.Scatter(x=t_events_failed["time_from_onset"], 
                        y=t_events_failed["description"],
                        name="failed",
                        mode="markers")
                    )  

    fig.update_layout(width=1100, height=800, title=title)
    return fig

fig = plot_interactive_testing_results(t_events, title="Test Test Test")
fig

# Tabular data extraction

In [ ]:
import os
import glob
import mne
from mne import Report
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from utils import (get_parent_dir, extract_lab_sec, loc_of_sep_lines, plot_seizure_horizontal, plot_seizure_vertical,
                                        extract_groups, 
                                        extract_ordered_groups, 
                                        save_plotly_to_html, 
                                        raw_to_df,
                                        shrink_df_to_tmax, create_results_folders, save_fig_to_disc,
                                        plot_interactive_subplot_with_table, extract_parameters_from_raw, 
                                        plot_eventcounts, plot_interactive_tables)
import plotly as py
import ipywidgets as widgets
import plotly.io as pio
from plotly.subplots import make_subplots

py.offline.init_notebook_mode(connected=True)
graph_sep_line_width = 5
plot_tmin = 5
plot_tmax = 60

win=False

edfs = glob.glob("../data/*.edf")
create_results_folders(edfs)

edfs

In [ ]:
df = dict() 
e_events = dict()
s_events = dict()
t_events = dict()
for e in edfs:
    print(f"Now processing file: {e}")
    raw = mne.io.read_raw(e, preload=True)
    df[e], onset = raw_to_df(raw, e)

    if win:
        e_events[e], s_events[e], t_events[e] = extract_ordered_groups(df[e], e.split("\\")[-1])
    else:
        e_events[e], s_events[e], t_events[e] = extract_ordered_groups(df[e], e.split("/")[-1]) 
    
    #save
    if win:
        csv_path = os.path.join("..", "results", e.split("\\")[-1].split(".")[0], "tables")
        e_file = e.split("\\")[-1].split(".")[0]
    else:
        csv_path = os.path.join("..", "results", e.split("/")[-1].split(".")[0], "tables")
        e_file = e.split("/")[-1].split(".")[0]
    csv_name = "All_data_" + e_file + ".csv"
    fname = os.path.join(csv_path, csv_name)
    df[e].to_csv(fname, sep="\t")
    csv_name = "EEG_data_" + e_file + ".csv"
    fname = os.path.join(csv_path, csv_name)
    e_events[e].to_csv(fname, sep="\t")
    csv_name = "Semiology_data_" + e_file + ".csv"
    fname = os.path.join(csv_path, csv_name)
    s_events[e].to_csv(fname, sep="\t")
    csv_name = "Testing_data_" + e_file + ".csv"
    fname = os.path.join(csv_path, csv_name)
    t_events[e].to_csv(fname, sep="\t")    

for idx, val in enumerate(df.keys()):
    if idx == 0:
        # all data vertical
        vconcat = df[val]
        # all data horizontal
        concat = df[val]
        source = "source_" + str(idx)
        concat[source] = val
        cols = list(concat)
        cols.insert(0, cols.pop(cols.index(source)))
        concat = concat.loc[:, cols]
        concat = concat.sort_values(by=["time_from_onset"])
        if "source" in concat.keys():
            concat.drop(columns=["source"], axis=1, inplace=True)
        concat["order_of_occurence"] = (1 + np.arange(len(concat.loc[:,"time_from_onset"])))
        # eeg, semio
        eeg_ga, semio_ga, test_ga = e_events[val], s_events[val], t_events[val]  # should be same keys as for e in edfs...

    if idx > 0:
        # all data vertical
        vnew_df = df[val]
        vconcat = pd.concat([vconcat, vnew_df], axis=0)
        # all data horizontal
        new_df = df[val]
        source = "source_" + str(idx)
        new_df[source] = val
        cols = list(new_df)
        cols.insert(0, cols.pop(cols.index(source)))
        new_df = new_df.loc[:, cols]
        if "source" in new_df.keys():
            new_df.drop(columns=["source"], axis=1, inplace=True)
        new_df["order_of_occurence"] = (1 + np.arange(len(new_df.loc[:,"time_from_onset"]))).astype(int)
        concat = pd.merge(concat, new_df, how="outer", on="description", suffixes=(" ", "  "))
        # eeg, semio
        ne, ns, nt = e_events[val], s_events[val], t_events[val]
        eeg_ga = pd.merge(eeg_ga, ne, how="outer", on="description", suffixes=(" ", "  ")) 
        semio_ga = pd.merge(semio_ga, ns, how="outer", on="description", suffixes=(" ", "  "))
        test_ga = pd.merge(test_ga, nt, how="outer", on="description", suffixes=(" ", "  "))

    idx += 1

if "source_0" in vconcat.keys():
    vconcat.drop(columns=["source_0"], axis=1, inplace=True)

# save grand averages
if win:
    eeg_ga.to_csv("..\\results\\grand_average\\tables\\EEG_grand_average.tsv", sep="\t")
    semio_ga.to_csv("..\\results\\grand_average\\tables\\Semiology_grand_average.tsv", sep="\t")
    test_ga.to_csv("..\\results\\grand_average\\tables\\Testing_grand_average.tsv", sep="\t")
    concat.to_csv("..\\results\\grand_average\\tables\\All_events_horizontal.tsv", sep="\t")
    vconcat.to_csv("..\\results\\grand_average\\tables\\All_events_vertical.tsv", sep="\t")

else:
    eeg_ga.to_csv("../results/grand_average/tables/EEG_grand_average.tsv", sep="\t")
    semio_ga.to_csv("../results/grand_average/tables/Semiology_grand_average.tsv", sep="\t")
    test_ga.to_csv("../results/grand_average/tables/Testing_grand_average.tsv", sep="\t")
    concat.to_csv("../results/grand_average/tables/All_events_horizontal.tsv", sep="\t")
    vconcat.to_csv("../results/grand_average/tables/All_data_grand_average.tsv", sep="\t")

In [ ]:
"""
# Grand average - order only
to_keep = [c for c in ga_h.columns if not c.lower().startswith("time")]
to_keep = [c for c in to_keep if not c.lower().startswith("unnam")]
if win:
    ga_h[to_keep].to_csv("..\\results\\grand_average\\tables\\All_events_order_only.csv", sep="\t")
else:
    ga_h[to_keep].to_csv("../results/grand_average/tables/All_events_order_only.csv", sep="\t")
oga_fig = plot_interactive_tables(ga_h[to_keep], EEG_ga[to_keep], semio_ga[to_keep], test_ga[to_keep])
source="grand_average"
cap = source + " --> Tabular data - order only"
report.add_htmls_to_section(oga_fig.to_html(full_html=False), section="grand_average", captions=cap)

"""

# Grand average VIZ

In [ ]:
import os
import glob
import mne
from mne import Report
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from utils import (get_parent_dir, extract_lab_sec, loc_of_sep_lines, plot_seizure_horizontal, plot_seizure_vertical,
                                        extract_groups, 
                                        extract_ordered_groups, 
                                        save_plotly_to_html, 
                                        raw_to_df,
                                        shrink_df_to_tmax, create_results_folders, save_fig_to_disc,
                                        plot_interactive_subplot_with_table, extract_parameters_from_raw, 
                                        plot_eventcounts, plot_interactive_tables)
import plotly as py
import ipywidgets as widgets
import plotly.io as pio
from plotly.subplots import make_subplots

py.offline.init_notebook_mode(connected=True)
graph_sep_line_width = 5
plot_tmin = 5
plot_tmax = 60

data = dict()
EEG = dict()
Semio = dict()
Test = dict()

data["grand_average"] = pd.read_csv("../results/grand_average/tables/All_data_grand_average.tsv", sep="\t")
data["grand_average"]

In [ ]:
def plot_gand_average_subplot_with_table(df=None, eeg=None, semio=None, testing=None, title=None):
    xaxis_title="Time in seconds (from seizure onset)"
    fig = make_subplots(rows=5, cols=1, shared_xaxes="all", 
                        specs=[[{"type": "table"}],
                                [{"type": "scatter"}],
                                [{"type": "scatter"}],
                                [{"type": "scatter"}],
                                [{"type": "scatter"}]],
                        subplot_titles=("Events", "EEG", "Semiology", "Testing", "All events"),
                        row_width=[0.1, 0.1, 0.1, 0.1, 0.8])
    # Add traces
    # data
    fig.add_trace(go.Table(
                        header=dict(
                                values=df.columns[2:], font=dict(size=10)),
                        cells=dict(
                            values=[df[i].tolist() for i in df.columns[2:]],
                            align="left")
                        ),
                        row=1, col=1)
    # scatter plots
    x_axis = df["time_from_onset"]
    y_axis = np.ones_like(x_axis)
    # eeg
    times, labels = extract_lab_sec(eeg)
    fig.add_trace(go.Scatter(x=times, y=y_axis,
                        mode='markers',                 #mode="markers+text"
                        hoverinfo="name+x+text",
                        name='EEG',
                        text=labels,
                        marker_symbol="diamond"), row=2, col=1)
    # semio
    times, labels = extract_lab_sec(semio)
    fig.add_trace(go.Scatter(x=times, y=y_axis,
                        mode='markers',
                        name='Semiology',
                        text=labels,
                        marker_symbol="x"), row=3, col=1)
    # testing
    times, labels = extract_lab_sec(testing)
    fig.add_trace(go.Scatter(x=times, y=y_axis,
                        mode='markers',
                        name='Testing',
                        text=labels,
                        marker_symbol="circle"), row=4, col=1)
    # grand average
    times, labels = extract_lab_sec(df)
    fig.add_trace(go.Scatter(x=times, y=y_axis,
                        mode='markers',
                        name='All events',
                        text=labels,
                        marker_symbol="hexagon2-open-dot"), row=5, col=1)

    fig.update_layout(title=title, yaxis_title="")
    fig.update_xaxes(rangeslider={"visible":True}, title={"text":xaxis_title}, row=5)
    fig.update_yaxes(visible=False, showticklabels=False)

    fig.update_layout(width=1500, height=1200)
    return fig


EEG["grand_average"], Semio["grand_average"], Test["grand_average"] = extract_ordered_groups(df=data["grand_average"])



plot_gand_average_subplot_with_table(df=data["grand_average"], eeg=EEG["grand_average"], semio=Semio["grand_average"], testing=Test["grand_average"], title="FooBar")